# CONTOURS OR SHAPE DETECTION

In [2]:
import cv2
import numpy as np


def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range (0, rows):
            for y in range (0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0,0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2:imgArray[x][y]= cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
          
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0,0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver


# Function to get contours
def getContours(img):
    contours,hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # the second argumemt retrieves the external/outer Contours
    for cnt in contours:
        area = cv2.contourArea(cnt)
        print(area)
        
        # Detect the minimum threshold of area so it does not detect any noise. # Detect the edges
        if area > 500:
            cv2.drawContours(imgContour, cnt, -1, (255,0,0), 3)
            # Calculate the curve length
            peri = cv2.arcLength(cnt, True)
            print(peri)
            
            # Approximate the coner points
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            print(len(approx))
            
            # Create object corners and put a bounding box around objects
            objCor = len(approx)
            
            x,y,w,h = cv2.boundingRect(approx)
            
            
            if objCor == 3: objectType = 'Tri' # Triangle
            elif objCor == 4:
                aspRatio = w/float(h)
                if aspRatio > 0.95 and aspRatio < 1.05: objectType = "square"
                else: objectType = "Rectangle"
            elif objCor > 4: objectType = "Circle"
            else: objectType = 'None'
            
            
            
            cv2.rectangle(imgContour, (x,y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(imgContour, objectType, (x+(w//2)-10, y+(h//2)-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 2) # To put text at the object centre



img  = cv2.imread('resources/shapes.png')
imgContour = img.copy()

# Image preprocessing, convert into grayscale
imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgBlur =  cv2.GaussianBlur(imgGray, (7,7), 1)
imgCanny = cv2.Canny(imgBlur, 50,50)

# Produce blank image
imgBlank = np.zeros_like(img)
getContours(imgCanny)





imgStack = stackImages(0.4, ([img, imgGray, imgBlur],
                             [imgCanny, imgContour, imgBlank]))

cv2.imshow('Stacked Images', imgStack)

cv2.waitKey(0)

9956.0
596.6518018245697
3
12994.0
427.3036036491394
8
10393.0
405.65685415267944
4
8283.0
399.65685415267944
4
7733.0
349.65685415267944
4
9589.0
403.65685415267944
4
15856.0
471.9310202598572
8
12535.0
528.1320307254791
3
17409.0
663.6568541526794
4
16406.0
479.5878744125366
8
14400.5
576.7178171873093
3


-1